## Introduction

The Restaurant Meals Program by CalFresh, California's Food Stamp Program, allows individuals who are homeless, elderly (age 51 and over), and disabled CalFresh households to use CalFresh benefits to purchase prepared meals to participating restaurants.

This dataset provides the macronutrient food information on different types of food available (as of March 1, 2021) from the following locations:

 - Carl’s Jr. 871 Marina Village Pkwy. Alameda, CA 94501
 
 - Carl’s Jr. 3770 Telegraph Ave. Oakland, CA 94690
 
 - Pizza Hut 2617 Decoto Rd. Union City, CA 94587
 
 - El Torero Taqueria 5801 International Blvd. Oakland CA 94621
 
 - Jamba Juice 3962 Mowry Ave. Fremont CA 94538

The data was constructed by creating a list of the available food items from each location. If available, we used the nutrition information provided by the restaurant. For well-known franchises, Carl's Jr, Pizza Hut and Jamba Juice, nutritional information was found through their website. For El Torero Taqueria, some food items lacked available nutrition information and had to be estimated through third-party websites such as SparkPeople.

We utilize the material from the earlier lecture on the subsistence diet problem, and in order to define the subsistence diet as a function of prices. 


First, we will install the tools we need from the requirements.txt file. Please make sure to include the requirements.txt file. Since we constructed the data, please refer to https://github.com/swhui/restaurant_data for the `price_only.csv` and `nutrition.csv` files.

In [1]:
#install requirements.txt 
# make sure to have requirements.txt
!pip install -r requirements.txt

  Using cached numpy-1.19.2-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
  Using cached pandas-1.1.3-cp38-cp38-manylinux1_x86_64.whl (9.3 MB)
  Using cached requests-2.25.0-py2.py3-none-any.whl (61 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.0
    Uninstalling pandas-1.2.0:
      Successfully uninstalled pandas-1.2.0
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before

Now, we will define the user and api key here:

In [2]:
user = "ligon"

# API key for Gov; substitute your own!
apikey = "inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN" # inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"

# File with private keys for relevant service account to authenticate
# and access google spreadsheets
serviceacct = {'ligon':'students-9093fa174318.json'}

## Input Data



The critical user input is a `pandas.DataFrame` with (at least)
these columns:

-   **Food:** Label used to identify food
-   **Price:** Price for quantity of food

### Using data from Google Sheets



### Compile data on food prices



We will now load in the data on the prices of the different items.

In [3]:
# import pandas library 
import pandas as pd
# read in the prices_only.csv
prices_only = pd.read_csv('price_only.csv')
# set the index to the food item and create a series object
Prices = (prices_only.set_index('food item')['price'])
Prices.head()

food item
acai primo bowl           9.29
vanilla blue sky bowl     9.29
chunky strawberry bowl    9.29
island pitaya bowl        9.29
ginger shot               3.79
Name: price, dtype: float64

### Look up nutritional information for foods



Now we have a list of foods with prices.  Do lookups on USDA database
to get nutritional information.



In [4]:
# read in the nutrition_only.csv file
nutrition_only = pd.read_csv('nutrition_only.csv').drop(['serving_size', 'cholesterol'], axis = 1)
# set the dataframe index as food item and rename columns appropriately
nutri = nutrition_only.set_index('food item').rename(columns = {
                            'calories'  : 'Energy',
                            'total_fat' : 'Total fat',
                            "saturated_fat": "Saturated fat",
                            'sodium' : 'Sodium, Na',
                            'total_carbohydrate' : 'Carbohydrate, by difference',
                            'dietary_fiber' : 'Fiber, total dietary',
                            'sugars' : 'Sugar',
                            'protein' : 'Protein'
                           })
# view the first 5 rows
nutri.head()

,Energy,Total fat,Saturated fat,"Sodium, Na","Carbohydrate, by difference","Fiber, total dietary",Sugar,Protein
food item,,,,,,,,
acai primo bowl,510.0,10.0,3.5,45.0,101.0,11.0,65.0,8.0
vanilla blue sky bowl,330.0,9.0,3.0,85.0,62.0,8.0,33.0,6.0
chunky strawberry bowl,580.0,16.0,2.5,135.0,94.0,11.0,50.0,21.0
island pitaya bowl,480.0,8.0,2.5,20.0,102.0,11.0,70.0,7.0
ginger shot,25.0,0.0,0.0,0.0,6.0,0.0,3.0,0.0


In [5]:
# transpose the nutri matrix
FoodNutrients = nutri.T
# view the first 5 rows
FoodNutrients.head()

food item,acai primo bowl,vanilla blue sky bowl,chunky strawberry bowl,island pitaya bowl,ginger shot,tumeric shot,wheatgrass shot,ginger lemon cayenne shot,ginger orange cayenne shot,"acai super-antioxidant smoothie, large",...,Lengua/Tongue Quesadilla,Carnitas/Fried Pork Quesadilla,Cabeza/Head Quesadilla,Pastor/Spicy Pork Quesadilla,Molida/Ground Beef Quesadilla,Pollo/Chicken Quesadilla,Tripas Quesadilla,Pescado/Fish Quesadilla,Camaron/Shrimp Quesadilla,Chile Verde o Roto Quesadilla
Energy,510.0,330.0,580.0,480.0,25.0,25.0,25.0,15.0,25.0,540.0,...,1020.0,397.4,990.0,1040.0,514.0,510.0,1070.0,750.0,400.0,460.0
Total fat,10.0,9.0,16.0,8.0,0.0,0.0,0.0,0.0,0.0,7.0,...,64.0,23.1,60.0,65.0,38.0,28.0,72.0,37.0,20.0,17.0
Saturated fat,3.5,3.0,2.5,2.5,0.0,0.0,0.0,0.0,0.0,2.0,...,7.0,9.7,6.0,7.0,13.0,12.0,39.0,16.0,7.0,8.0
"Sodium, Na",45.0,85.0,135.0,20.0,0.0,0.0,0.0,0.0,0.0,125.0,...,2010.0,920.5,2110.0,2260.0,845.0,1210.0,2420.0,1660.0,500.0,860.0
"Carbohydrate, by difference",101.0,62.0,94.0,102.0,6.0,6.0,5.0,4.0,5.0,110.0,...,57.0,13.5,57.0,58.0,42.0,38.0,48.0,69.0,33.0,41.0


## Dietary Requirements



We&rsquo;ve figured out some foods we can buy, the nutritional content of
those foods, and  the price of the foods.  Now we need to say
something about nutritional requirements.   Our data for this is based
on  US government recommendations available at
[https://www.dietaryguidelines.gov/sites/default/files/2019-05/2015-2020_Dietary_Guidelines.pdf](https://www.dietaryguidelines.gov/sites/default/files/2019-05/2015-2020_Dietary_Guidelines.pdf).

I&rsquo;ve put some of these data into a google spreadsheet at
[https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/](https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/).

Unfortunately, not all of the nutrition requirements that we need are in this document, so we had to obtain the data using the FDA recommendations and calculation from other requirements. In particular, we had to add in data to obtain the requiremenets for sugar, total fats, and saturated fats.

In [6]:
# import the read_sheets
from eep153_tools import read_sheets

# Obtain the dietary requirements spreadsheet link as a string
DRIs = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

# Define *minimums*
diet_min = read_sheets(DRIs,json_creds=serviceacct[user],sheet='diet_minimums').set_index('Nutrition')

# Define *maximums*
diet_max = read_sheets(DRIs,json_creds=serviceacct[user],sheet='diet_maximums').set_index('Nutrition')

In [7]:
# import numpy
import numpy as np

add_requirements = diet_min.iloc[[0, 1, 2, 5]].T
add_requirements = add_requirements[1:]

# create the sugar requirements
add_requirements['Sugar'] =  (add_requirements['Energy'].astype(float) * .10) * 25/100 * 1 

# obtain an array of the energies/calories
energies = np.array(add_requirements['Energy'].tolist())
# obtain data on fat requirements
DRI_fat_guide = np.array([.4,.35,.35,.35,.35,.35,.35,.35,.35,.35,.35,.35,.35])
# get total fats
totfat = np.around((energies * DRI_fat_guide) * (1/9), 2)
# get saturated fats
sat_fats = np.around((energies * .10) * (1/9), 2)
# create columns in add_requirements
add_requirements['Total fat'] = totfat
add_requirements['Saturated fat'] = sat_fats

# add_requirements[['Energy', 'Total fat', 'Saturated fat', 'Carbohydrate, by difference', 
#           'Fiber, total dietary', 'Sugar', 'Protein']]

#add_requirements

Now, we will show the diet minimum requirements for different age groups.

In [8]:
diet_min = add_requirements[['Energy', 'Carbohydrate, by difference', 
                      'Fiber, total dietary', 'Protein']]
diet_min = diet_min.transpose()
# Minimums Energy, Carbohydrate, by difference, Fiber, total dietary, Protein
diet_min

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000,1200,1400,1600,1800,1800,2200,2000,2400,1800,2200,1600,2000
"Carbohydrate, by difference",130,130,130,130,130,130,130,130,130,130,130,130,130
"Fiber, total dietary",14,16.8,19.6,22.4,25.2,25.2,30.8,28,33.6,25.2,30.8,22.4,28
Protein,13,19,19,34,34,46,52,46,56,46,56,46,56


Now, we will show the diet maximum requirements for different age groups.

In [9]:
# Maximums are Sodium, Total fat, Saturated fat, Sugar
diet_max = diet_max.T
diet_max = diet_max.iloc[1:]
diet_max['Total fat'] = add_requirements['Total fat']
diet_max['Saturated fat'] = add_requirements['Saturated fat']
diet_max['Sugar'] = add_requirements['Sugar']
diet_max = diet_max.T
diet_max

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
"Sodium, Na",1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300
Total fat,44.44,46.67,54.44,62.22,70,70,85.56,77.78,93.33,70,85.56,62.22,77.78
Saturated fat,11.11,13.33,15.56,17.78,20,20,24.44,22.22,26.67,20,24.44,17.78,22.22
Sugar,25,30,35,40,45,45,55,50,60,45,55,40,50


## Putting it together



Here we take the different pieces of the puzzle we&rsquo;ve developed and
put them together in the form of a linear program we can solve.
Recall that the mathematical problem we&rsquo;re trying to solve is
$$
    \min_x p'x
$$
such that
$$
     Ax \geq b
$$
If we buy a bag of groceries with quantities given by $x$, the total
cost of the bag of groceries is the inner product of prices and
quantities.  Since we&rsquo;ve converted our units above, this gives us a
vector of prices where quantities are all in 100 g or ml units.

The following code block defines a function



In [10]:
# define the subsistence problem
from  scipy.optimize import linprog as lp
import numpy as np

def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    #p = Prices.apply(lambda x:x.magnitude).dropna()
    p = Prices

    # Compile list that we have both prices and nutritional info for; drop if either missing
    #use = list(set(p.index.tolist()).intersection(FoodNutrients.columns.tolist()))
    #p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[diet_min.index]

    Amax = Aall.loc[diet_max.index]

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,-Amax])

    b = pd.concat([diet_min,-diet_max]) # Note sign change for max constraints

    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    result.diet = pd.Series(result.x,index=p.index)

    return result

## Using =solve_subsistence_problem= to analyze diet for females 51+



Let&rsquo;s choose a particular group, females of age 51+, and solve the subsistence problem for them:



In [11]:
# We wish to solve the subsistence problem  for females of age 51 and over
group = 'F 51+'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet as a proportion of the food item:")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for F 51+ is $9.64 per day.


Diet as a proportion of the food item:
food item
purely carrot juice, large                                     0.390522
Marinara Dipping Sauce (3 oz), Stuffed Pizza Rollers (Each)    0.140378
Marinara Dipping Sauce (3 oz)                                  0.140378
Lengua/Tongue Taco                                             1.399942
Chile Verde o Roto Taco                                        2.698890
dtype: float64


With the following nutritional outcomes of interest:
                                 Outcome Recommendation
Nutrition                                              
Energy                       1600.000000           1600
Carbohydrate, by difference   181.121823            130
Fiber, total dietary           22.400000           22.4
Protein                        76.822488             46
Sodium, Na                   2300.000000           2300
Total fat                      62.220000          62.22
Saturated fat                 

As prices change, we should expect the minimum cost diet to also change.  The code below creates a graph which changes prices away from the base case food, which we chose to be the purely carrot juice (large) at a time, and plots changes in total diet cost.

In [12]:
import cufflinks as cf
cf.go_offline()

scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

tol = 10**(-9)

cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

Price_response={}
for s in scale:
    cost = {}
    for i,p in enumerate(Prices):
        my_p = Prices.copy()
        my_p[i] = p*s
        result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
        
        cost[Prices.index[i]] = (result.fun/cost0)
    Price_response[np.log(s)] = cost

Price_response = pd.DataFrame(Price_response).T

Price_response = Price_response.loc[:, (Price_response > tol).sum()>0]

np.log(Price_response).iplot(xTitle='Log price change',yTitle='Proportional Cost Change',
                    layout = cf.Layout(height = 1000, width = 2000) )


/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning:

The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.

Error importing optional module geopandas
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/_plotly_utils/optional_imports.py", line 30, in get_module
    return import_module(name)
  File "/opt/conda/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>

In [13]:
ReferenceGood = "purely carrot juice, large"

scale = [1.,1.1,1.2,1.3,1.4,1.5,2,4,8]


cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
    diet[np.log(my_p[ReferenceGood]) ] = result.diet

tol = 10e-6
    
Diet_response = pd.DataFrame(diet).T
np.log(Diet_response.loc[:,(Diet_response>tol).sum()>0]).iplot(xTitle='Purely Carrot Juice, Large Log Price',
                                                       yTitle='Log Price')

Interestingly enough, as the price of the purely carrot juice changes, the fish burrito seems to be more important to the diet.

In [14]:
# We wish to solve the subsistence problem  for males of age 51 and over
group = 'M 51+'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet as a proportion of the food item:")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 51+ is $12.39 per day.


Diet as a proportion of the food item:
food item
purely carrot juice, large    0.582760
Sierra Mist                   0.174309
Lengua/Tongue Taco            1.973379
Chile Verde o Roto Taco       2.629008
dtype: float64


With the following nutritional outcomes of interest:
                                 Outcome Recommendation
Nutrition                                              
Energy                       2000.000003           2000
Carbohydrate, by difference   221.984035            130
Fiber, total dietary           28.000000             28
Protein                        84.013976             56
Sodium, Na                   2300.000009           2300
Total fat                      77.780001          77.78
Saturated fat                  16.571837          22.22
Sugar                          49.836078             50


Constraining nutrients are:
['Energy', 'Fiber, total dietary', 'Sodium, Na', 'Total fat']


## Effects of Price Changes on Subsistence Diet Cost



As prices change, we should expect the minimum cost diet to also change.  The code below creates a graph which changes prices away from the base case food, which we chose to be the purely carrot juice (large) at a time, and plots changes in total diet cost.



In [15]:
scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

tol = 10**(-9)

cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

Price_response={}
for s in scale:
    cost = {}
    for i,p in enumerate(Prices):
        my_p = Prices.copy()
        my_p[i] = p*s
        result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
        
        cost[Prices.index[i]] = (result.fun/cost0)
    Price_response[np.log(s)] = cost

Price_response = pd.DataFrame(Price_response).T

Price_response = Price_response.loc[:, (Price_response > tol).sum()>0]

np.log(Price_response).iplot(xTitle='Log price change',yTitle='Proportional Cost Change',
                    layout = cf.Layout(height = 1000, width = 2000) )


The code below creates a graph which changes prices just for *one* food,
  and traces out the effects of this change on all the foods consumed.



In [16]:
ReferenceGood = "purely carrot juice, large"

scale = [1.,1.1,1.2,1.3,1.4,1.5,2,4,8]


cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
    diet[np.log(my_p[ReferenceGood]) ] = result.diet

tol = 10e-6
    
Diet_response = pd.DataFrame(diet).T
np.log(Diet_response.loc[:,(Diet_response>tol).sum()>0]).iplot(xTitle='Purely Carrot Juice, Large Log Price',
                                                       yTitle='Log Price')

Interestingly enough, as the price of the purely carrot juice changes, the fish burrito seems to be more important to the diet.